#Lab 6
### part 1

In [15]:
#1.	Import the libraries

import pandas as pd
import numpy as np
from ast import literal_eval


In [16]:
#2.	Load the dataset into a datafrme

df = pd.read_csv('/content/ccai422_lab06_part1_data.csv')
#Print all the features (or columns) of the DataFrame
df.columns


<ipython-input-16-fd8657fd3c20>:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/ccai422_lab06_part1_data.csv')


Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [17]:
#3.	Now, let's reduce our DataFrame to only contain features that we need for our model:

#Only keep those features that we require
df = df[['title','genres', 'release_date', 'runtime', 'vote_average', 'vote_count']]
df.head()


,title,genres,release_date,runtime,vote_average,vote_count
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0


In [18]:
#4.	Next, let us extract the year of release from our release_date feature

#Convert release_date into pandas datetime format
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
#Extract year from the datetime
df['year'] = df['release_date'].apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)


In [19]:
#5.	Our year feature is still an object and is riddled with NaT values, which are a type of null value used by Pandas. Let's convert these values to an integer, 0, and convert the datatype of the year feature into int. To do this, we will define a helper function, convert_int, and apply it to the year feature:

#Helper function to convert NaT to 0 and all other years to integers.
def convert_int(x):
    try:
        return int(x)
    except:
        return 0
#Apply convert_int to the year feature
df['year'] = df['year'].apply(convert_int)


In [20]:
#6.	We do not require the release_date feature anymore. So let's go ahead and remove it:

#Drop the release_date column
df = df.drop('release_date', axis=1)


In [21]:
#7.	Upon preliminary inspection, we can observe that the genres are in a format that looks like a JSON object (or a Python dictionary). Let us take a look at the genres object of one of our movies:
#Print genres of the first movie
df.iloc[0]['genres']


"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [22]:
#8.	We can observe that the output is a stringified dictionary. In order for this feature to be usable, it is important that we convert this string into a native Python dictionary. Fortunately, Python gives us access to a function called literal_eval (available in the ast library) which does exactly that. literal_eval parses any string passed into it and converts it into its corresponding Python object:

#Import the literal_eval function from ast
from ast import literal_eval
#Define a stringified list and output its type
a = "[1,2,3]"
print(type(a))
#Apply literal_eval and output type
b = literal_eval(a)
print(type(b))


<class 'str'>
<class 'list'>


In [23]:
#9.	We now have all the tools required to convert the genres feature into the Python dictionary format. Also, each dictionary represents a genre and has two keys: id and name. However, for this exercise (as well as all subsequent exercises), we only require the name. Therefore, we shall convert our list of dictionaries into a list of strings, where each string is a genre name:

#Convert all NaN into stringified empty lists
df['genres'] = df['genres'].fillna('[]')
#Apply literal_eval to convert to the list object
df['genres'] = df['genres'].apply(literal_eval)
#Convert list of dictionaries to a list of strings
df['genres'] = df['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df.head()


,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[Animation, Comedy, Family]",81.0,7.7,5415.0,1995
1,Jumanji,"[Adventure, Fantasy, Family]",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[Romance, Comedy]",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[Comedy, Drama, Romance]",127.0,6.1,34.0,1995
4,Father of the Bride Part II,[Comedy],106.0,5.7,173.0,1995


In [24]:
#10.	Printing the head of the DataFrame should show you a new genres feature, which is a list of genre names. However, we're still not done yet. The last step is to explode the genres column. In other words, if a particular movie has multiple genres, we will create multiple copies of the movie, with each movie having one of the genres. For example, if there is a movie called Just Go With It that has romance and comedy as its genres, we will explode this movie into two rows. One row will be Just Go With It as a romance movie. The other will be a comedy movie:

#Create a new feature by exploding genres
s = df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
#Name the new feature as 'genre'
s.name = 'genre'
#Create a new dataframe gen_df which by dropping the old 'genres' feature and adding the new 'genre'.
gen_df = df.drop('genres', axis=1).join(s)
#Print the head of the new gen_df
gen_df.head()


,title,runtime,vote_average,vote_count,year,genre
0,Toy Story,81.0,7.7,5415.0,1995,Animation
0,Toy Story,81.0,7.7,5415.0,1995,Comedy
0,Toy Story,81.0,7.7,5415.0,1995,Family
1,Jumanji,104.0,6.9,2413.0,1995,Adventure
1,Jumanji,104.0,6.9,2413.0,1995,Fantasy


In [25]:
#11.	The build_chart function: We are finally in a position to write the function that will act as our recommender, these are three main steps:
#a.	Get user input on their preferences
#b.	Extract all movies that match the conditions set by the user
#c.	Calculate the values of m and C for only these movies and proceed to build the chart as in the previous section
#Therefore, the build_chart function will accept only two inputs: our gen_df DataFrame and the percentile used to calculate the value of m. By default, let's set this to 80%, or 0.8:

def build_rec(gen_df, percentile=0.8):
    #Ask for preferred genres
    print("Input preferred genre")
    genre = input()
    #Ask for lower limit of duration
    print("Input shortest duration")
    low_time = int(input())
    #Ask for upper limit of duration
    print("Input longest duration")
    high_time = int(input())
    #Ask for lower limit of timeline
    print("Input earliest year")
    low_year = int(input())
    #Ask for upper limit of timeline
    print("Input latest year")
    high_year = int(input())
    #Define a new movies variable to store the preferred movies. Copy the contents of gen_df to movies
    movies = gen_df.copy()
    #Filter based on the condition
    movies = movies[(movies['genre'] == genre) &
                    (movies['runtime'] >= low_time) &
                    (movies['runtime'] <= high_time) &
                    (movies['year'] >= low_year) &
                    (movies['year'] <= high_year)]
    #Compute the values of C and m for the filtered movies
    C = movies['vote_average'].mean()
    m = movies['vote_count'].quantile(percentile)
    #Only consider movies that have higher than m votes. Save this in a new dataframe q_movies
    q_movies = movies.copy().loc[movies['vote_count'] >= m]
    #Calculate score using the IMDB formula
    q_movies['score'] = q_movies.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average'])
                                       + (m/(m+x['vote_count']) * C)
                                       ,axis=1)
    #Sort movies in descending order of their scores
    q_movies = q_movies.sort_values('score', ascending=False)
    return q_movies


In [32]:
#12.	Time to put the model into action. We want recommendations for animated movies between 30 minutes and 2 hours in length, and released anywhere between 1990 and 2005. Let's see the results:

build_rec(gen_df).head()


Input preferred genre
Comedy
Input shortest duration
81
Input longest duration
104
Input earliest year
1995
Input latest year
1995


,title,runtime,vote_average,vote_count,year,genre,score
0,Toy Story,81.0,7.7,5415.0,1995,Comedy,7.666018
68,Friday,91.0,7.0,513.0,1995,Comedy,6.804245
38,Clueless,97.0,6.9,828.0,1995,Comedy,6.781394
177,Mallrats,94.0,6.8,400.0,1995,Comedy,6.598772
328,Tommy Boy,97.0,6.8,321.0,1995,Comedy,6.560741


### part 2

1.	Download the dataset file that is provided on the Blackboard and browse its content
2.	Read the data into a dataframe


In [28]:
import pandas as pd

# Load Part 2 data into a DataFrame for digital cameras
df_cameras = pd.read_csv('/content/ccai422_lab06_part2_data.csv')

# Display the first few rows and columns to understand the structure
df_cameras.head(), df_cameras.columns


(   no  price  mpix opt_zoom  LCD_size movies sound waterproof
 0  p1    148   8.0       4x       2.5     no    no        yes
 1  p2    182   8.0       5x       2.7    yes   yes         no
 2  p3    189   8.0      10x       2.5    yes   yes         no
 3  p4    196  10.0      12x       2.7    yes    no        yes
 4  p5    151   7.1       3x       3.0    yes   yes         no,
 Index(['no', 'price', 'mpix', 'opt_zoom', 'LCD_size', 'movies', 'sound',
        'waterproof'],
       dtype='object'))

3.	[2.5 marks] Modify the function build_rec that is provided to you in the previous part to ask the user for the lowest and highest price as well as the usage
Note : Usage can be either Sports (Sports   waterproof & sound) or Fashion (Fashion  mpix>=8 & opt-zoom>5)
Test with this Requirements:
a.	The price should be lower than 300
b.	The camera should be suited for sports photography


In [29]:
# Modified recommendation function for Part 2
def build_camera_rec(df, max_price, usage_type):
    # Make a copy of the DataFrame to avoid modifying the original data
    cameras = df.copy()

    # Apply the initial filtering based on price and usage requirements
    if usage_type == "Sports":
        # Filter for Sports: requires 'waterproof' and 'sound'
        cameras = cameras[(cameras['price'] < max_price) &
                          (cameras['waterproof'] == 'yes') &
                          (cameras['sound'] == 'yes')]
    elif usage_type == "Fashion":
        # Filter for Fashion: requires mpix >= 8 and opt_zoom > 5x
        cameras = cameras[(cameras['price'] < max_price) &
                          (cameras['mpix'] >= 8) &
                          (cameras['opt_zoom'].apply(lambda x: int(x.replace('x', '')) > 5))]

    return cameras

# Test the function with given requirements:
# Price < 300 and camera suited for sports
filtered_cameras = build_camera_rec(df_cameras, max_price=300, usage_type="Sports")
filtered_cameras.head()


,no,price,mpix,opt_zoom,LCD_size,movies,sound,waterproof
7,p8,278,9.1,10x,3.0,yes,yes,yes


4.	[2.5 marks] Based on P4, the user provided the following critiques:
a.	Cheaper
b.	Better Zooming
Modify the function to apply these critiques and recommend items accordingly.


In [30]:
# Enhanced recommendation function with critiques for "Cheaper" and "Better Zooming"
def build_camera_rec_with_critiques(df, max_price, usage_type):
    # Make a copy of the DataFrame to avoid modifying the original data
    cameras = df.copy()

    # Apply the initial filtering based on price and usage requirements
    if usage_type == "Sports":
        # Filter for Sports: requires 'waterproof' and 'sound'
        cameras = cameras[(cameras['price'] < max_price) &
                          (cameras['waterproof'] == 'yes') &
                          (cameras['sound'] == 'yes')]
    elif usage_type == "Fashion":
        # Filter for Fashion: requires mpix >= 8 and opt_zoom > 5x
        cameras = cameras[(cameras['price'] < max_price) &
                          (cameras['mpix'] >= 8) &
                          (cameras['opt_zoom'].apply(lambda x: int(x.replace('x', '')) > 5))]

    # Apply additional critiques: further reduce price and increase zoom
    max_price -= 50  # Applying "cheaper" critique by reducing price threshold
    cameras = cameras[cameras['price'] < max_price]

    # Applying "better zooming" critique by prioritizing higher optical zoom
    cameras = cameras[cameras['opt_zoom'].apply(lambda x: int(x.replace('x', '')) > 8)]

    return cameras

# Test the function with the critiques applied
enhanced_cameras = build_camera_rec_with_critiques(df_cameras, max_price=300, usage_type="Sports")
enhanced_cameras.head()


""


In [35]:
# Adjusted recommendation function with moderated critiques for "Cheaper" and "Better Zooming"
#To present a feasible solution, we can advise the user that available recommendations may not meet both critiques fully, and instead, provide options based on individual criteria (e.g., focusing on either lower price or better zoom separately)
def build_camera_rec_with_moderate_critiques(df, max_price, usage_type):
    # Make a copy of the DataFrame to avoid modifying the original data
    cameras = df.copy()

    # Apply the initial filtering based on price and usage requirements
    if usage_type == "Sports":
        # Filter for Sports: requires 'waterproof' and 'sound'
        cameras = cameras[(cameras['price'] < max_price) &
                          (cameras['waterproof'] == 'yes') &
                          (cameras['sound'] == 'yes')]
    elif usage_type == "Fashion":
        # Filter for Fashion: requires mpix >= 8 and opt_zoom > 5x
        cameras = cameras[(cameras['price'] < max_price) &
                          (cameras['mpix'] >= 8) &
                          (cameras['opt_zoom'].apply(lambda x: int(x.replace('x', '')) > 5))]

    # Apply moderated critiques: slightly reduce price and increase zoom
    max_price -= 25  # Less strict "cheaper" critique
    cameras = cameras[cameras['price'] < max_price]

    # Less strict "better zooming" critique: set minimum zoom to 6x
    cameras = cameras[cameras['opt_zoom'].apply(lambda x: int(x.replace('x', '')) > 6)]

    return cameras

# Test the function with moderated critiques
moderated_cameras = build_camera_rec_with_moderate_critiques(df_cameras, max_price=300, usage_type="Sports")
moderated_cameras.head()


""


# The End
      .-""""-.
     / -    - \
    |  O    O  |
    |     >    |
     \  \__/  /
      `-.....-`
                /
        /|\\   /
        / | \\/
          |
        / \\
        /   \
